---------------------------------------------------------------------------
### Step by step guide to create a Tensorflow 2.0, CUDA enabled, Python environment within WSL2
---------------------------------------------------------------------------

In [ ]:
# Getting Started

This quick start guide assumes
    # - A CUDA enabled NVIDIA GPU is present.
        # - Check the CUDA Compatibility list for more info - https://developer.nvidia.com/cuda-gpus
    # - A CUDA enabled NVIDIA driver is installed in Windows
    # - An Ubuntu WSL2 Distro is up and running in a supported Windows build
    # - Network/Internet is available within Windows and the WSL2 environment
    # - A user was created during the deployment of Ubuntu
    # - Basic understanding of ubuntu Linux command Line, PowerShell and Python (Nothing heavy is needed)
    
More info
    # TFML = Tensorflow Machine Learning
    # My Distro is named 'ubuntu-tfml' which I use in the examples
    # My Conda environment is named 'tfml' which I use in the examples

In [ ]:
# Set the Default user in /etc/wsl.conf
# This must be done as root
sudo su
echo -e '# My TFML Environment \n[boot] \nsystemd=true \n \n[user] \ndefault=userx \n' > /etc/wsl.conf

In [ ]:
# Restart Ubuntu
# Exit Ubuntu then in PowerShell run the following
wsl --shutdown
wsl -d ubuntu-tfml

In [ ]:
# Update/upgrade with apt
sudo apt update && sudo apt upgrade

In [ ]:
# Create the folder structure in \home\user directory
cd ~
mkdir {_admin,_temp,docs,downloads,env,projects,env/base,env/tfml}

In [ ]:
# Download latest version of Miniconda
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -P ~/downloads/

In [ ]:
# Install Miniconda
bash ~/downloads/Miniconda3-latest-Linux-x86_64.sh

In [ ]:
# Cleanup install files
rm ~/downloads/Miniconda3-latest-Linux-x86_64.sh

In [ ]:
# Restart Ubuntu
# Exit Ubuntu then in PowerShell run the following
wsl --shutdown
wsl -d ubuntu-tfml

In [ ]:
# Update Conda
conda update conda

In [ ]:
# Save the environments package info in 'env/base'
conda env export > ~/env/base/conda-env-export.yaml

In [ ]:
# Create and activate the tfml environment
wget https://raw.githubusercontent.com/BrandXX/tfml-env-pub/master/tfml/conda-env-export.yaml -P ~/env/tfml/
conda env create --file=env/tfml/conda-env-export.yaml
conda activate tfml

---------------------------------------------------------------------------
### Tensorflow 2 config
---------------------------------------------------------------------------

In [ ]:
# Config Conda environment activation scripts and set GPU Lib env var
mkdir -p $CONDA_PREFIX/etc/conda/activate.d
echo 'CUDNN_PATH=$(dirname $(python -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)"))' >> $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
echo 'export LD_LIBRARY_PATH=$CONDA_PREFIX/lib/:$CUDNN_PATH/lib:$LD_LIBRARY_PATH' >> $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
mkdir -p $CONDA_PREFIX/lib/nvvm/libdevice/
cp -p $CONDA_PREFIX/lib/libdevice.10.bc $CONDA_PREFIX/lib/nvvm/libdevice/
echo 'export XLA_FLAGS=--xla_gpu_cuda_data_dir=$CONDA_PREFIX/lib' >> $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
source $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh

---------------------------------------------------------------------------
### Testing the environment
---------------------------------------------------------------------------

In [ ]:
# Verify Test01:
python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

# Verify Test01 successful output
    # Don't worry too much about Warnings or Information messages. The primary goal of the test is to list the GPU in the last line.

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Verify Test02
python3 -c "import tensorflow.compat.v1 as tf; tf.test.is_gpu_available()"

# Verify Test02 successful output
    # Don't worry too much about Warnings or Information messages. The primary goal of the test is to list the GPU details in the last line.

2023-08-24 22:48:09.936172: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:0 with 21286 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9

In [ ]:
# Verify Test03
python3 -c "import tensorflow.compat.v1 as tf; tf.enable_eager_execution(tf.ConfigProto(log_device_placement=True)); print(tf.add([1.0, 2.0], [3.0, 4.0]))"

# Verify Test03 successful output
    # Don't worry too much about Warnings or Information messages. The primary goal is to show a 1-dimensional TensorFlow tensor with two elements both stored as 32-bit floating-point numbers.
tf.Tensor([4. 6.], shape=(2,), dtype=float32)

---------------------------------------------------------------------------
### Finishing up
---------------------------------------------------------------------------

In [ ]:
# Save Package Lists for tfml environment
conda env export > ~/env/tfml/conda-env-export.yaml


---------------------------------------------------------------------------
### Install and configure VScode
---------------------------------------------------------------------------

In [ ]:
# No need to install VScode in your WSL environment. Install VScode on your host Windows machine.
# Once installed run code . from the Linux terminal
code .

# Recommended VScode Extensions
Visual Studio Code Remote Development Extension Pack
URL: https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.vscode-remote-extensionpack
Description: The Remote Development extension pack allows you to open any folder in a container, on a remote machine, or in the Windows Subsystem for Linux (WSL) and take advantage of VS Code's full feature set.

Name: Python Development Extensions Pack
URL: https://marketplace.visualstudio.com/items?itemName=demystifying-javascript.python-extensions-pack
Description: This extension pack packages some of the most popular and some of my favorite extensions needed for fullstack Python development or web development.

Name: Git Extension Pack
URL: https://marketplace.visualstudio.com/items?itemName=donjayamanne.git-extension-pack
Description: This extension pack packages some of the most popular (and some of my favorite) Git extensions.

---------------------------------------------------------------------------
### Backup the WSL environment
---------------------------------------------------------------------------

In [ ]:
# Backup the WSL Distro via PowerShell
wsl --export ubuntu-tfml path/ubuntu-tfml-00.tar

---------------------------------------------------------------------------
### Additional Useful Info
---------------------------------------------------------------------------

GIT global config

In [ ]:
# Set Global Git Username and Email
git config --global user.name "Your Name"
git config --global user.email "youremail@yourdomain.com"

In [ ]:
# Confirm the Git config changes
git config --list

Saving Package info

In [ ]:
# Recommended
conda env export > ~/env/tfml/conda-env-export.yaml

# This can be used with pip install -r requirements.txt
pip freeze > ~/env/tfml/requirements.txt 

# Extras
conda list --explicit > ~/env/tfml/package-list-explicit_tf2.txt
conda list > ~/env/tfml/package-list-simple_tf2.txt

Check versions and info

In [ ]:
# Check Python version
python3 --version

In [ ]:
# Check Tensorflow version
python3 -c "import tensorflow as tf; print('TensorFlow version:', tf.__version__)"

# <OR>

python3 -c "import tensorflow as tf; print(tf.__version__)"

In [ ]:
# Check Keras version
python -c 'import keras; print(keras.__version__)'

In [ ]:
# Nvidia environment info
nvidia-smi